In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys 
sys.path.insert(0, '/content/drive/MyDrive/CBN_Automate')
%cd /content/drive/MyDrive/CBN_Automate

import pandas as pd
import Resource  as res
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from googleapiclient import discovery
import psycopg2
import psycopg2.extras as extras



scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name("cbn-file-Automate.json",scope)
service = discovery.build('sheets', 'v4', credentials=credentials)

/content/drive/MyDrive/CBN_Automate


###  Input URL'S

In [ ]:
### Input file Url 
urls = ["https://docs.google.com/spreadsheets/d/1goS1Ghcvkp0wkcnIlCqEbAEIpwxRjf3hZcCynf0LoaM/edit?urlBuilderDomain=smartsetter.io#gid=2090963965"]

In [ ]:
#To get spreadsheet informations like spreadsheet file name and sheet name
spreadsheet_id = []
sheet_id = []
file_name = []
sheet_name = []
for sid in urls:
    spread_id = sid.split('/')
    spreadsheet_id.append(spread_id[5])
    sh_id = sid.split('=')
    sheet_id.append(sh_id[-1])
    sheet_metadata = service.spreadsheets().get(spreadsheetId=spread_id[5]).execute()
    file_name.append(sheet_metadata.get('properties').get('title'))
    properties = sheet_metadata.get('sheets')
    
    for  item in properties:
        if item.get("properties").get('sheetId') == int(sh_id[-1]):
            sh_name = (item.get("properties").get('title'))
            sheet_name.append(sh_name)

In [ ]:
#For reading all the files and creating dynamic variable for each dataframe
column_length = []
for url in range(len(urls)):
    
    globals()[f"df{url}"] = res.read_dataframe(urls[url])
    
    print(globals()[f"df{url}"].shape)
    column_length.append(globals()[f"df{url}"].shape[1])

    #globals()[f"df{url}"]["Assign"]  = None  

(1594, 18)


In [ ]:
#for checking duplicates
for url in range(len(urls)):
    
    
    license = res.license_finder(globals()[f"df{url}"].columns)

    globals()[f"df{url}"]["Comments"]  = None 
    
    if len(license)>0:

        globals()[f"df{url}"] = res.license_count(globals()[f"df{url}"],license)

        globals()[f"df{url}"] = res.license_count(globals()[f"df{url}"],"Found_Mobile")

        #globals()[f"df{url}"] = res.duplicate_check(globals()[f"df{url}"])

        globals()[f"df{url}"] = res.comment(globals()[f"df{url}"])

        globals()[f"df{url}"].drop(['license_id_count','Found_Mobile_count'],axis= 1 , inplace= True)

        none_index = globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"].isna()].index

        if len(none_index):
          globals()[f"df{url}"].loc[none_index,"Comments"] = None

        none_index = globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"] == 'None'].index

        if len(none_index):
          globals()[f"df{url}"].loc[none_index,"Comments"] = None       

        none_index = globals()[f"df{url}"][globals()[f"df{url}"]["Found_Mobile"] == 'Not Found'].index

        if len(none_index):
          globals()[f"df{url}"].loc[none_index,"Comments"] = None    

Taking  license_id as license


In [ ]:
#To export to google sheets

albha = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","AA","AB","AC"]
for url in range(len(urls)):
    
    Data = globals()[f"df{url}"][["Comments"]]
    
    na_index = Data[Data["Comments"].isna()].index
    if len(na_index) > 0:
        Data.loc[na_index,"Comments"] = None
        

    
    
    sheet_data_list = []
    for index in range(len(Data)):
        sheet_data_list.append([str(Data.loc[index,'Comments'])])
    


    data = [['Comments']]
    request = service.spreadsheets().values().append(spreadsheetId=spreadsheet_id[url], 
                                                     range=sheet_name[url]+"!"+albha[column_length[url]]+str(1)+":"+albha[column_length[url]+1]+str(1), valueInputOption="USER_ENTERED", 
                                                     insertDataOption="OVERWRITE", body={"values":data})

    response = request.execute()
    
        
   



    data = sheet_data_list
    request = service.spreadsheets().values().append(spreadsheetId=spreadsheet_id[url], 
                                                     range=sheet_name[url]+"!"+albha[column_length[url]]+str(2)+":"+albha[column_length[url]+1]+str(2), valueInputOption="USER_ENTERED", 
                                                     insertDataOption="OVERWRITE", body={"values":data})

    response = request.execute()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
